In [31]:
import pandas as pd
import zipfile
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import squarify
import plotly.express as px
from scipy.stats import chi2_contingency
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [32]:
zip_path = '../BD/Supermercado.zip' 
csv_filename = 'Supermercado.csv'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(csv_filename) as f:
        df = pd.read_csv(f)


In [33]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [34]:
week_days = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}

df['day'] = df['order_dow'].map(week_days)

In [35]:
df.isnull().sum()

order_id                       0
user_id                        0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    124342
product_id                     0
add_to_cart_order              0
reordered                      0
department_id                  0
department                     0
product_name                   0
day                            0
dtype: int64

Dado que el objetivo es crear un aloritmo de clasificacion, se eliminan las filas nulas del dataset.

In [36]:
df = df.dropna()

Finalmente, se valida de que efectivamente se hayan eliminado los valores nulos.

In [37]:
df.isnull().sum()

order_id                  0
user_id                   0
order_number              0
order_dow                 0
order_hour_of_day         0
days_since_prior_order    0
product_id                0
add_to_cart_order         0
reordered                 0
department_id             0
department                0
product_name              0
day                       0
dtype: int64

Se valida de que no existan valores duplicados en el dataset.

#### Ventas segun Dia y Momento del Dia

Con el objetivo de realizar una segmentación de horarios por parte del día, se procede a realizar un mapeo de horas, agrupandolas por los momentos: "Morning", "Afternoon", "Night" y "Down".

In [38]:
def order_time(x):
    if x in[6, 7, 8, 9, 10, 11, 12]:
        return 'Morning'
    if x in[13, 14, 15, 16, 17]:
        return 'Afternoon'
    if x in[18, 19, 20, 21, 22]:
        return 'Night'
    if x in[23, 24, 0, 1, 2, 3, 4, 5]:
        return 'Dawn'
    return x

Se almacena la el valor de momento del día en la variable "order_time_list".

In [39]:
df['order_time_list']=df['order_hour_of_day'].apply(order_time)
df.sample(5)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name,day,order_time_list
264613,1881414,186780,15,6,8,19.00,48,1,1,14,breakfast,breakfast bars pastries,Sunday,Morning
1154982,2102968,906,4,0,14,30.00,49,7,0,12,meat seafood,packaged poultry,Monday,Afternoon
1131870,146135,63446,21,2,11,2.00,123,3,1,4,produce,packaged vegetables fruits,Wednesday,Morning
1789722,885266,73692,61,5,16,2.00,67,5,1,20,deli,fresh dips tapenades,Saturday,Afternoon
457436,78051,649,46,6,20,13.00,9,16,1,9,dry goods pasta,pasta sauce,Sunday,Night


#### Segmentacion de Clientes por Cantidad de Ordenes

Se maneja la hipótesis de que el comportamiento de los clientes varia según la cantidad de ordenes o compras que realizan a lo largo del período de tiempo de análisis.
Por tal motivo, se plantea segmentar en grupos de clientes por cantidad de ordenes realizadas.

In [40]:
max_order_numbers = df.groupby('user_id')['order_number'].max().reset_index()
max_order_numbers.rename(columns={'order_number': 'max_order'}, inplace=True)

df = df.merge(max_order_numbers, on='user_id', how='left')

El enfoque que se plantea es el de generar 5 grupos de clientes según la cantidad de compras realizadas, comenzando con ordenes entre 1 y 20, y finalizando con el último grupo que posee clientes que realizaron entre 81 y 100 pedidos (siendo 100 el máximo de ordenes realizadas por clientes registradas en el dataset).

In [41]:
def order_number_group(x):
    if x <= 20:
        x = '1-20 order'
    elif x >20 and x <=40:
        x = '21-40 order'
    elif x >40 and x <=60:
        x = '41-60 order'
    elif x >60 and x <=80:
        x = '61-80 order'
    else:
        x = '81-100 order'
    return x

Luego de realizar el mapeo de grupos de clientes, se procede a almacenar la variable en la columna 'order_number_group'. 

In [42]:
df['order_number_group']=df['max_order'].apply(order_number_group)
df.sample(5)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name,day,order_time_list,max_order,order_number_group
989377,2484094,171360,16,4,14,2.00,92,2,1,18,babies,baby food formula,Friday,Afternoon,53,41-60 order
609579,637734,154311,9,1,10,14.00,107,11,1,19,snacks,chips pretzels,Tuesday,Morning,9,1-20 order
687468,1538964,110141,64,5,9,4.00,24,5,1,4,produce,fresh fruits,Saturday,Morning,64,61-80 order
131882,3174790,95273,10,3,11,15.00,24,3,0,4,produce,fresh fruits,Thursday,Morning,42,41-60 order
1022322,2991393,55896,10,4,9,7.00,37,4,1,1,frozen,ice cream ice,Friday,Morning,10,1-20 order


#### One_Hot 

Tomamos todas las variables categóricas para realizar el one hot encoding 

In [43]:
df_one_hot = pd.get_dummies(df, columns=['order_number_group', 'department', 'product_name', 'day'], dtype=int, drop_first=True)
df_one_hot.sample(5)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,...,product_name_vitamins supplements,product_name_water seltzer sparkling water,product_name_white wines,product_name_yogurt,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
549227,1093715,55096,34,5,12,8.00,123,1,1,4,...,0,0,0,0,0,1,0,0,0,0
717425,70299,186537,18,4,19,28.00,67,22,1,20,...,0,0,0,0,0,0,0,0,0,0
1202840,2268158,54979,55,6,21,1.00,115,2,1,7,...,0,1,0,0,0,0,1,0,0,0
837769,694826,163057,3,2,16,10.00,91,12,0,16,...,0,0,0,0,0,0,0,0,0,1
14541,3047882,170970,22,6,13,8.00,120,3,1,16,...,0,0,0,1,0,0,1,0,0,0


In [44]:
columns_to_group = [col for col in df_one_hot.columns if 'product_name_' in col or 'department_' in col]

df_one_hot_ag_counts = df_one_hot.groupby('order_id')[columns_to_group].sum()

df_other_columns = df_one_hot.drop(columns=columns_to_group).drop_duplicates(subset='order_id').set_index('order_id')

df_one_hot_ag = df_other_columns.join(df_one_hot_ag_counts).reset_index()

df_one_hot_ag.sample(5)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_time_list,...,product_name_spreads,product_name_tea,product_name_tofu meat alternatives,product_name_tortillas flat bread,product_name_trail mix snack mix,product_name_trash bags liners,product_name_vitamins supplements,product_name_water seltzer sparkling water,product_name_white wines,product_name_yogurt
16052,2584518,154914,5,1,11,30.00,83,1,1,Morning,...,0,0,0,0,0,0,0,0,0,1
90286,2643734,107536,5,4,7,24.00,112,1,0,Morning,...,0,0,0,0,0,0,0,0,0,0
140631,835806,132004,16,4,11,7.00,83,1,1,Morning,...,0,0,0,0,0,0,0,0,0,0
84751,1982314,13467,40,4,11,6.00,115,1,1,Morning,...,0,0,0,0,0,0,0,1,0,0
178464,3241704,10083,31,1,11,2.00,34,1,0,Morning,...,0,0,0,0,0,0,0,0,0,0


Verifico

In [50]:
order_id_to_check = 1226918

original_data = df[df['order_id'] == order_id_to_check]
print("Datos en el dataset original para order_id", order_id_to_check)
print(original_data)

aggregated_data = df_one_hot_ag[df_one_hot_ag['order_id'] == order_id_to_check]
print("\nDatos en el dataset agrupado para order_id", order_id_to_check)
aggregated_data.head()


Datos en el dataset original para order_id 1226918
         order_id  user_id  order_number  order_dow  order_hour_of_day  \
1785760   1226918     7103             3          0                 20   
1785761   1226918     7103             3          0                 20   
1785762   1226918     7103             3          0                 20   
1785763   1226918     7103             3          0                 20   
1785764   1226918     7103             3          0                 20   
1785765   1226918     7103             3          0                 20   
1785766   1226918     7103             3          0                 20   
1785767   1226918     7103             3          0                 20   
1785768   1226918     7103             3          0                 20   
1785769   1226918     7103             3          0                 20   
1785770   1226918     7103             3          0                 20   
1785771   1226918     7103             3          0          

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_time_list,...,product_name_spreads,product_name_tea,product_name_tofu meat alternatives,product_name_tortillas flat bread,product_name_trail mix snack mix,product_name_trash bags liners,product_name_vitamins supplements,product_name_water seltzer sparkling water,product_name_white wines,product_name_yogurt
176962,1226918,7103,3,0,20,12.00,37,1,1,Night,...,0,0,0,1,0,0,0,2,0,1


#### Se exporta el DF con los cambios realizados

In [ ]:
df_one_hot_ag.to_csv('../BD/Supermercado_onehot.csv', index=False)

zip_filename = '../BD/Supermercado_onehot.zip'
csv_filename = '../BD/Supermercado_onehot.csv'

with zipfile.ZipFile(zip_filename, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_filename, arcname='Supermercado_onehot.csv')